# Import

In [1]:
%load_ext memory_profiler

In [2]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:
# Misc
import os
import json
import joblib
import warnings

# Data management
import numpy as np
import pandas as pd

# Sound treatments
import librosa
import soundfile as sf
from scipy import signal
import resampy

# Model
import tensorflow as tf

# TRILL
import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()
assert tf.executing_eagerly()
import tensorflow_hub as hub

# EfficientNetB0
from keras.applications.efficientnet import EfficientNetB0
from keras.applications.efficientnet import preprocess_input

# Meta model
from sklearn.ensemble import RandomForestClassifier

## Metrics
import tensorflow_addons as tfa
from tensorflow_addons.layers.netvlad import NetVLAD

# Environment

In [4]:
# Inactivate warnings
warnings.filterwarnings('ignore')

# Display Tensorlfow version
print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 2.6.0


In [5]:
#DATA_PATH = '/kaggle/input/birdclef-2022/'
#WORKING_PATH = '/kaggle/working/'
#TRILL_PATH = '/kaggle/input/ziptrill/'
#VGGISH_PATH = '/kaggle/input/vggishfull/'
#MODEL_PATH = '/kaggle/input/models/'

DATA_PATH = './data/'
WORKING_PATH = './working/stacking/'
TRILL_PATH = './working/stacking/trill/'
VGGISH_PATH = './working/stacking/'
MODEL_PATH = './working/stacking/'

In [6]:
t_off = False
e_off = False
v_off = False

# Data load

In [7]:
# Load meta data
train_meta = pd.read_csv(DATA_PATH + 'train_metadata.csv')

# Load scored birds
with open(DATA_PATH + 'scored_birds.json') as sbfile:
    scored_birds = json.load(sbfile)
    
# Focus on 21 scored classes
labels = list(train_meta[train_meta['primary_label'].isin(scored_birds)]['primary_label'].unique())
labels

['akiapo',
 'aniani',
 'apapan',
 'barpet',
 'crehon',
 'elepai',
 'ercfra',
 'hawama',
 'hawcre',
 'hawgoo',
 'hawhaw',
 'hawpet1',
 'houfin',
 'iiwi',
 'jabwar',
 'maupar',
 'omao',
 'puaioh',
 'skylar',
 'warwhe1',
 'yefcan']

# Model

In [8]:
def create_cnn(model_name):
    if model_name == 'trill':
        model = tf.keras.models.Sequential()
        model.add(tf.keras.Input((80000,)))

        trill_layer = hub.KerasLayer(
            handle=TRILL_PATH,
            trainable=False,
            arguments={'sample_rate': int(16000)},
            output_key='embedding',
            output_shape=[None, 2048]
        )

        model.add(trill_layer)
        model.add(NetVLAD(num_clusters=8))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dense(256, activation='relu'))
        model.add(tf.keras.layers.Dense(21, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(l=1e-5)))

    elif model_name == 'efficientnetb0':
        base_model = EfficientNetB0(include_top=False, input_shape=(224, 224, 3), weights=None, pooling='avg')
        dense = tf.keras.layers.Dense(142, activation='relu')(base_model.output)
        outputs = tf.keras.layers.Dense(21, activation='sigmoid')(dense)
        base_model.trainable = False
        model = tf.keras.models.Model(inputs=base_model.input, outputs=outputs)
        
    elif model_name == 'vggish':
        '''base_model, _, _ = vgk.get_embedding_model(hop_duration=0.25)   
        dense = Dense(128, activation='relu')(base_model.output)
        outputs = Dense(21, activation='sigmoid')(dense)      
        base_model.trainable = False
        model = Model(inputs=base_model.input, outputs=outputs)'''
        
        model = tf.keras.models.load_model(VGGISH_PATH + 'VGGish_full.h5')
        
    return model

# Submission

## Feature extraxction

### Trill

In [9]:
# Sound noise reduction
def f_high_trill(y, sr):
    b, a = signal.butter(10, 1000/(sr/2), btype='highpass')
    yf = signal.lfilter(b, a, y)
    return yf

In [10]:
def extractFeatures_trill(y, sr):
    # Sound noise reduction
    y = f_high_trill(y, sr)
    # Resample
    y = librosa.resample(y, sr, 16000)

    return y

### EfficientNetB0

In [11]:
class conf:
    # Preprocessing settings
    sampling_rate = 44100
    n_mels = 224
    hop_length = 494
    n_fft = n_mels * 10
    fmin = 20
    fmax = 16000
    
    # Model parameters
    num_rows = 224
    num_columns = 224
    num_channels = 3

In [12]:
def audio_to_melspectrogram(audio):
    spectrogram = librosa.feature.melspectrogram(audio,
                                                 sr=conf.sampling_rate,
                                                 n_mels=conf.n_mels,
                                                 hop_length=conf.hop_length,
                                                 n_fft=conf.n_fft,
                                                 fmin=conf.fmin,
                                                 fmax=conf.fmax)
    spectrogram = librosa.power_to_db(spectrogram)
    spectrogram = spectrogram.astype(np.float32)
    return spectrogram

In [13]:
def mono_to_color(X, eps=1e-6, mean=None, std=None):
    """
    Converts a one channel array to a 3 channel one in [0, 255]
    Arguments:
        X {numpy array [H x W]} -- 2D array to convert
    Keyword Arguments:
        eps {float} -- To avoid dividing by 0 (default: {1e-6})
        mean {None or np array} -- Mean for normalization (default: {None})
        std {None or np array} -- Std for normalization (default: {None})
    Returns:
        numpy array [3 x H x W] -- RGB numpy array
    """
    X = np.stack([X, X, X], axis=-1)

    # Standardize
    mean = mean or X.mean()
    std = std or X.std()
    X = (X - mean) / (std + eps)

    # Normalize to [0, 255]
    _min, _max = X.min(), X.max()

    if (_max - _min) > eps:
        V = np.clip(X, _min, _max)
        V = 255 * (V - _min) / (_max - _min)
        V = V.astype(np.uint8)
    else:
        V = np.zeros_like(X, dtype=np.uint8)

    return V

In [14]:
def extractFeatures_EfficientNetB0(y, sr):
    # Extract features
    feat = audio_to_melspectrogram(y)
    feat = mono_to_color(feat)
    feat = feat.astype(np.uint8)
    
    # EfficientNet preprocess
    feat = preprocess_input(feat)
    
    X = np.empty((1, conf.num_rows, conf.num_columns, conf.num_channels))
    x_features = feat.tolist()
    X[0] = np.array(x_features)
        
    return X

### VGGish

In [15]:
# Sound noise reduction
def f_high_VGGish(y,sr):
    b,a = signal.butter(10, 2000/(sr/2), btype='highpass')
    yf = signal.lfilter(b,a,y)
    return yf

In [16]:
def waveform_to_examples(data, sample_rate):
    # Convert to mono.
    if len(data.shape) > 1:
        data = np.mean(data, axis=1)
    # Resample to the rate assumed by VGGish.
    if sample_rate != 16000:
        data = resampy.resample(data, sample_rate, 16000)

    # Compute log mel spectrogram features.
    log_mel = log_mel_spectrogram(
        data,
        audio_sample_rate=16000,
        log_offset=0.01,
        window_length_secs=0.025,
        hop_length_secs=0.010,
        num_mel_bins=64,
        lower_edge_hertz=125,
        upper_edge_hertz=7500)

    # Frame features into examples.
    features_sample_rate = 1.0 / 0.010
    example_window_length = int(round(0.96 * features_sample_rate))
    example_hop_length = int(round(0.96 * features_sample_rate))
    log_mel_examples = frame(
        log_mel,
        window_length=example_window_length,
        hop_length=example_hop_length)
    return log_mel_examples

In [17]:
def log_mel_spectrogram(data,
                        audio_sample_rate=8000,
                        log_offset=0.0,
                        window_length_secs=0.025,
                        hop_length_secs=0.010,
                        **kwargs):
    window_length_samples = int(round(audio_sample_rate * window_length_secs))
    hop_length_samples = int(round(audio_sample_rate * hop_length_secs))
    fft_length = 2 ** int(np.ceil(np.log(window_length_samples) / np.log(2.0)))
    spectrogram = stft_magnitude(
        data,
        fft_length=fft_length,
        hop_length=hop_length_samples,
        window_length=window_length_samples)
    mel_spectrogram = np.dot(spectrogram, spectrogram_to_mel_matrix(
        num_spectrogram_bins=spectrogram.shape[1],
        audio_sample_rate=audio_sample_rate, **kwargs))
    return np.log(mel_spectrogram + log_offset)

In [18]:
def frame(data, window_length, hop_length):
    num_samples = data.shape[0]
    num_frames = 1 + int(np.floor((num_samples - window_length) / hop_length))
    shape = (num_frames, window_length) + data.shape[1:]
    strides = (data.strides[0] * hop_length,) + data.strides
    return np.lib.stride_tricks.as_strided(data, shape=shape, strides=strides)

In [19]:
def stft_magnitude(signal, fft_length,
                   hop_length=None,
                   window_length=None):
    frames = frame(signal, window_length, hop_length)
    # Apply frame window to each frame. We use a periodic Hann (cosine of period
    # window_length) instead of the symmetric Hann of np.hanning (period
    # window_length-1).
    window = periodic_hann(window_length)
    windowed_frames = frames * window
    return np.abs(np.fft.rfft(windowed_frames, int(fft_length)))

In [20]:
def periodic_hann(window_length):
    return 0.5 - (0.5 * np.cos(2 * np.pi / window_length *
                               np.arange(window_length)))

In [21]:
def spectrogram_to_mel_matrix(num_mel_bins=20,
                              num_spectrogram_bins=129,
                              audio_sample_rate=8000,
                              lower_edge_hertz=125.0,
                              upper_edge_hertz=3800.0):
    nyquist_hertz = audio_sample_rate / 2.
    if lower_edge_hertz < 0.0:
        raise ValueError("lower_edge_hertz %.1f must be >= 0" %
                         lower_edge_hertz)
    if lower_edge_hertz >= upper_edge_hertz:
        raise ValueError("lower_edge_hertz %.1f >= upper_edge_hertz %.1f" %
                         (lower_edge_hertz, upper_edge_hertz))
    if upper_edge_hertz > nyquist_hertz:
        raise ValueError("upper_edge_hertz %.1f is greater than Nyquist %.1f" %
                         (upper_edge_hertz, nyquist_hertz))
    spectrogram_bins_hertz = np.linspace(
        0.0, nyquist_hertz, num_spectrogram_bins)
    spectrogram_bins_mel = hertz_to_mel(spectrogram_bins_hertz)
    # The i'th mel band (starting from i=1) has center frequency
    # band_edges_mel[i], lower edge band_edges_mel[i-1], and higher edge
    # band_edges_mel[i+1].  Thus, we need num_mel_bins + 2 values in
    # the band_edges_mel arrays.
    band_edges_mel = np.linspace(hertz_to_mel(lower_edge_hertz),
                                 hertz_to_mel(upper_edge_hertz), num_mel_bins + 2)
    # Matrix to post-multiply feature arrays whose rows are num_spectrogram_bins
    # of spectrogram values.
    mel_weights_matrix = np.empty((num_spectrogram_bins, num_mel_bins))
    for i in range(num_mel_bins):
        lower_edge_mel, center_mel, upper_edge_mel = band_edges_mel[i:i + 3]
        # Calculate lower and upper slopes for every spectrogram bin.
        # Line segments are linear in the *mel* domain, not hertz.
        lower_slope = ((spectrogram_bins_mel - lower_edge_mel) /
                       (center_mel - lower_edge_mel))
        upper_slope = ((upper_edge_mel - spectrogram_bins_mel) /
                       (upper_edge_mel - center_mel))
        # .. then intersect them with each other and zero.
        mel_weights_matrix[:, i] = np.maximum(0.0, np.minimum(lower_slope,
                                                              upper_slope))
    # HTK excludes the spectrogram DC bin; make sure it always gets a zero
    # coefficient.
    mel_weights_matrix[0, :] = 0.0
    return mel_weights_matrix

In [22]:
def hertz_to_mel(frequencies_hertz):
    return 1127.0 * np.log(
        1.0 + (frequencies_hertz / 700.0))

In [23]:
def extractFeatures_VGGish(y, sr):
    # Sound noise reduction
    y = f_high_VGGish(y, sr)
    
    feat = waveform_to_examples(y, sr)
        
    return feat

## Process

In [24]:
test_path = DATA_PATH + '/test_soundscapes/'
files = [f.split('.')[0] for f in sorted(os.listdir(test_path))]
print('Number of test soundscapes:', len(files))

Number of test soundscapes: 1


In [25]:
%memit ids = []
%memit dd_t = {}
%memit dd_e = {}
%memit dd_v = {}

for f in files:
    file_path = test_path + f + '.ogg'

    # Load audio file
    audio, sr = librosa.load(file_path)

    # Get number of samples for 5 seconds
    buffer = 5 * sr
    block_min = 5 * sr

    samples_total = len(audio)
    samples_wrote = 0
    counter = 1

    while samples_wrote < samples_total:
        # check if the buffer is not exceeding total samples
        if buffer > (samples_total - samples_wrote):
            buffer = samples_total - samples_wrote

        # Block treatment
        block = audio[samples_wrote: (samples_wrote + buffer)]
        if block.shape[0] < (block_min):
            listofzeros = np.array([0] * (block_min - block.shape[0]))
            block = np.hstack([block, listofzeros])
           
        # row id
        segment_end = counter * 5
        row_id = f + '_bird_' + str(segment_end)
        ids.append(row_id)

        # Features extraction
        if not t_off:
            block_trill = extractFeatures_trill(block, sr)
            %memit dd_t[row_id] = block_trill.reshape(1, -1)
            #joblib.dump(block_trill.reshape(1, -1), WORKING_PATH + 'blocks/trill_' + row_id + '.jl')
            
        if not e_off:
            block_EfficientNetB0 = extractFeatures_EfficientNetB0(block, sr)
            %memit dd_e[row_id] = block_EfficientNetB0
            #joblib.dump(block_EfficientNetB0, WORKING_PATH + 'blocks/EfficientNetB0_' + row_id + '.jl')
        
        if not v_off:
            block_VGGish = extractFeatures_VGGish(block, sr)
            %memit dd_v[row_id] = block_VGGish.reshape(1, 480, 64, 1)
            #joblib.dump(block_VGGish.reshape(1, 480, 64, 1), WORKING_PATH + 'blocks/VGGish_' + row_id + '.jl')
            
        counter += 1
        samples_wrote += buffer

peak memory: 374.57 MiB, increment: 0.12 MiB
peak memory: 374.57 MiB, increment: 0.00 MiB
peak memory: 374.57 MiB, increment: 0.00 MiB
peak memory: 374.57 MiB, increment: 0.00 MiB
peak memory: 399.96 MiB, increment: 0.00 MiB
peak memory: 406.71 MiB, increment: 0.01 MiB
peak memory: 407.48 MiB, increment: 0.00 MiB
peak memory: 407.48 MiB, increment: 0.00 MiB
peak memory: 409.25 MiB, increment: 0.03 MiB
peak memory: 409.54 MiB, increment: 0.00 MiB
peak memory: 409.29 MiB, increment: 0.00 MiB
peak memory: 411.40 MiB, increment: 0.00 MiB
peak memory: 411.66 MiB, increment: 0.00 MiB
peak memory: 411.41 MiB, increment: 0.00 MiB
peak memory: 413.59 MiB, increment: 0.00 MiB
peak memory: 413.59 MiB, increment: 0.00 MiB
peak memory: 413.44 MiB, increment: 0.00 MiB
peak memory: 415.93 MiB, increment: 0.00 MiB
peak memory: 415.86 MiB, increment: -0.06 MiB
peak memory: 415.61 MiB, increment: 0.00 MiB
peak memory: 417.87 MiB, increment: 0.00 MiB
peak memory: 417.96 MiB, increment: 0.00 MiB
peak memo

In [26]:
# This is where we will store our results
pred = {'row_id': [], 'target': []}

In [27]:
# Make prediction for each chunk
# Each scored bird gets a random value in our case
# since we don't actually have a model
for i in range(len(ids)):        
    chunk_end_time = (i + 1) * 5
    for bird in labels:

        # This is our random prediction score for this bird
        score = np.random.uniform()

        # Assemble the row_id which we need to do for each scored bird
        row_id = ids[i] + '_' + bird + '_' + str(chunk_end_time)

        # Put the result into our prediction dict and
        # apply a "confidence" threshold of 0.5
        pred['row_id'].append(row_id)
        pred['target'].append(True if score > 0.5 else False)

In [28]:
# Make a new data frame and look at some results        
results = pd.DataFrame(pred, columns = ['row_id', 'target'])

# Quick sanity check
print(results.head(21)) 

                                   row_id  target
0    soundscape_453028782_bird_5_akiapo_5    True
1    soundscape_453028782_bird_5_aniani_5    True
2    soundscape_453028782_bird_5_apapan_5    True
3    soundscape_453028782_bird_5_barpet_5   False
4    soundscape_453028782_bird_5_crehon_5    True
5    soundscape_453028782_bird_5_elepai_5   False
6    soundscape_453028782_bird_5_ercfra_5   False
7    soundscape_453028782_bird_5_hawama_5   False
8    soundscape_453028782_bird_5_hawcre_5   False
9    soundscape_453028782_bird_5_hawgoo_5    True
10   soundscape_453028782_bird_5_hawhaw_5    True
11  soundscape_453028782_bird_5_hawpet1_5   False
12   soundscape_453028782_bird_5_houfin_5   False
13     soundscape_453028782_bird_5_iiwi_5   False
14   soundscape_453028782_bird_5_jabwar_5    True
15   soundscape_453028782_bird_5_maupar_5    True
16     soundscape_453028782_bird_5_omao_5   False
17   soundscape_453028782_bird_5_puaioh_5    True
18   soundscape_453028782_bird_5_skylar_5    True


In [29]:
# Convert our results to csv
results.to_csv(WORKING_PATH + 'submission.csv', index=False)    

In [30]:
raise

RuntimeError: No active exception to reraise

In [ ]:
batchsize = 12
steps = len(ids) / batchsize
calibration_limit = 0.1

print('batchsize:', batchsize)
print('steps:', steps)
print('calibration_limit:', calibration_limit)

### TRILL features

In [ ]:
# TRILL features
m_Trill = create_cnn('trill')
m_Trill.load_weights(MODEL_PATH + 'trill.h5')
m_Trill.compile(optimizer=tf.keras.optimizers.Adam(),
                loss='binary_crossentropy',
                metrics=[tfa.metrics.F1Score(name='f1macro', num_classes=len(labels), average='macro')])

count_prediction = 0

for i in range(0, int(steps)+1):
    temp = ids[i*batchsize:(i*batchsize)+batchsize]

    X = np.empty((len(temp), 80000))
    count_batch = 0
    for batch in temp:
        feat = joblib.load(WORKING_PATH + 'blocks/trill_' + batch + '.jl')
        X[count_batch] = feat
        count_batch += 1

    # Prediction
    if not t_off:
        if X.shape[0] != 0:
            pred_trill = m_Trill.predict(X)

            for prediction in pred_trill:
                count_prediction += 1
                joblib.dump(prediction, WORKING_PATH +
                            'blocks/features/trill_' + str(count_prediction) + '.jl')
    else:
        for j in range(0, count_batch):
            count_prediction += 1
            joblib.dump([1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                         0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                         0], WORKING_PATH +
                        'blocks/features/trill_' + str(count_prediction) + '.jl')

# Kill objects
del temp
del X
del m_Trill

### EfficientNetB0 features

In [ ]:
# EfficientNetB0 features
m_EfficientNetB0 = create_cnn('efficientnetb0')
m_EfficientNetB0.load_weights(MODEL_PATH + 'EfficientNetB0.h5')
m_EfficientNetB0.compile(optimizer=tf.keras.optimizers.Adam(),
                         loss='binary_crossentropy',
                         metrics=[tfa.metrics.F1Score(name='f1macro', num_classes=len(labels), average='macro')])

count_prediction = 0

for i in range(0, int(steps)+1):
    temp = ids[i*batchsize:(i*batchsize)+batchsize]

    X = np.empty((len(temp), 224, 224, 3))
    count_batch = 0
    for batch in temp:
        feat = joblib.load(
            WORKING_PATH + 'blocks/EfficientNetB0_' + batch + '.jl')
        X[count_batch] = feat
        count_batch += 1

    # Prediction
    if not e_off:
        if X.shape[0] != 0:
            pred_EfficientNetB0 = m_EfficientNetB0.predict(X)

            for prediction in pred_EfficientNetB0:
                count_prediction += 1
                joblib.dump(prediction, WORKING_PATH +
                            'blocks/features/EfficientNetB0_' + str(count_prediction) + '.jl')
    else:
        for j in range(0, count_batch):
            count_prediction += 1
            joblib.dump([1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                         0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                         0], WORKING_PATH +
                        'blocks/features/EfficientNetB0_' + str(count_prediction) + '.jl')

# Kill objects
del temp
del X
del m_EfficientNetB0

### VGGish features

In [ ]:
# VGGish features
m_VGGish = create_cnn('vggish')
m_VGGish.compile(optimizer=tf.keras.optimizers.Adam(),
                 loss='binary_crossentropy',
                 metrics=[tfa.metrics.F1Score(name='f1macro', num_classes=len(labels), average='macro')])

count_prediction = 0

for i in range(0, int(steps)+1):
    temp = ids[i*batchsize:(i*batchsize)+batchsize]

    X = np.empty((len(temp), 480, 64, 1))
    count_batch = 0
    for batch in temp:
        feat = joblib.load(WORKING_PATH + 'blocks/VGGish_' + batch + '.jl')
        X[count_batch] = feat
        count_batch += 1

    # Prediction
    if not v_off:
        if X.shape[0] != 0:
            pred_VGGish = m_VGGish.predict(X)

            for prediction in pred_VGGish:
                count_prediction += 1
                joblib.dump(prediction, WORKING_PATH +
                            'blocks/features/VGGish_' + str(count_prediction) + '.jl')
    else:
        for j in range(0, count_batch):
            count_prediction += 1
            joblib.dump([1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                         0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                         0], WORKING_PATH +
                        'blocks/features/VGGish_' + str(count_prediction) + '.jl')

# Kill objects
del temp
del X
del m_VGGish

### Meta

In [ ]:
X = np.empty((count_prediction, 63))

for i in range(1, count_prediction+1):
    # Prediction
    pred_trill = joblib.load(
        WORKING_PATH + 'blocks/features/trill_' + str(i) + '.jl')
    pred_EfficientNetB0 = joblib.load(
        WORKING_PATH + 'blocks/features/EfficientNetB0_' + str(i) + '.jl')
    pred_VGGish = joblib.load(
        WORKING_PATH + 'blocks/features/VGGish_' + str(i) + '.jl')

    res_list = []
    for j in pred_trill:
        res_list.append(j)
    for j in pred_EfficientNetB0:
        res_list.append(j)
    for j in pred_VGGish:
        res_list.append(j)
        
    X[i-1] = np.array(res_list).reshape(1, -1)

# construct meta dataset
meta_X = pd.DataFrame(X)

In [ ]:
# Prediction
meta_model = joblib.load(MODEL_PATH + 'meta_model.jl')

pred_meta = meta_model.predict_proba(meta_X)

del meta_X
del meta_model

In [ ]:
data = []

for j in range(0, count_prediction):
    label_indexes = []
    for i in range(0, 21):
        if pred_meta[i][j][1] >= calibration_limit:
            label_indexes.append(i)
 
    print('label_indexes', label_indexes)

    for b in scored_birds:
        row_id = ids[j].replace('bird', b)
        target = False
        for label_index in label_indexes:
            if labels[label_index] == b:
                target = True
        data.append([row_id, target])

submission_df = pd.DataFrame(data, columns=['row_id', 'target'])
submission_df.head(21)

In [ ]:
submission_df.to_csv(WORKING_PATH + 'submission.csv', index=False)